In [6]:
import asyncio
from data_project.gsheets import DateSheet
from datetime import date, timedelta, datetime
import pandas as pd


target_date = (date.today() - timedelta(days=1))

async def create_sheet():
    return DateSheet('[RP_CN] Revenue Report', date=target_date.isoformat())

task_sheet = asyncio.create_task(create_sheet())
await task_sheet
mysheet = task_sheet.result()

# Filter Data

In [7]:
# Change your sheet
mysheet.change_sheet("data", headers_row=2)
sheet = mysheet.worksheet

# Get all values from the sheet
data = sheet.get_all_values()

# Assuming headers are in the second row (index 1)
headers = data[1]
data = data[2:]

# Create a DataFrame
df = pd.DataFrame(data, columns=headers)

# Filter rows where "Fan Leoi 1" column has the value "Bou Shek"
filtered_df = df[df['分類1'] == '寶石']

# Display the filtered DataFrame
print(filtered_df)

/Users/serenayu/Documents/data-project/data_project/gsheets.py:81: UserWarning: No columns named '日期' or 'date'.
  warnings.warn("No columns named '日期' or 'date'.")


       serverPrefix      userId       Create Time Package ID Package Name  \
0              1109  1109018958  2024-03-31 23:41        225        60日冕石   
11             1109  1109017731  2024-03-31 19:24        225        60日冕石   
12             1108  1108014351  2024-03-28 18:15        225        60日冕石   
13             1107  1107001330  2024-03-22 16:32        225        60日冕石   
51             1109  1109012153   2024-03-31 0:51        225        60日冕石   
...             ...         ...               ...        ...          ...   
364519         1143  1143013532  2024-07-25 16:42        225        60日冕石   
364586         1143  1143011596  2024-07-25 10:41        225        60日冕石   
364588         1143  1143011596  2024-07-25 10:41        225        60日冕石   
364589         1143  1143011596  2024-07-25 10:41        225        60日冕石   
364610         1143  1143015258  2024-07-25 22:36        225        60日冕石   

       Package Type price    Purchase Time 分類1 分類2    USD         Unique ID

# Insert Data

In [8]:
# select the target sheet
mysheet = DateSheet('nightless_city_test', date=target_date.isoformat())
mysheet.change_sheet('工作表1')
sheet = mysheet.worksheet
data = filtered_df.copy()
data = data.iloc[: 10]  # only update the first 10 rows

# define the relation of columns
column_names = {
    "userId": "userId",
    "serverPrefix": "serverPrefix",
    "Create Time": "CT",
}

# rename the col of data
data.columns = [column_names.get(c, c).lower() for c in data.columns]

# sort & filter columns
for header in mysheet.headers:
    if header not in data.columns:
        data.loc[:, header] = None
data = data[mysheet.headers]

# sort rows
# data['ct'] = data['ct'].str.replace(" ", "T").apply(datetime.fromisoformat)  # convert str into datetime
data.sort_values("ct", ascending=True, inplace=True)

# compute the cell to paste
cell = f"A{len(sheet.col_values(1)) + 1}"

# update the target sheet
sheet.update(cell, data.values.tolist())

/Users/serenayu/Documents/data-project/data_project/gsheets.py:81: UserWarning: No columns named '日期' or 'date'.
  warnings.warn("No columns named '日期' or 'date'.")
/var/folders/ct/v7_skn0d1qj1fx21m526kmwm0000gn/T/ipykernel_4255/1454590208.py:32: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update(cell, data.values.tolist())


{'spreadsheetId': '1SeKA_q4lj2AUgakSebhzlzW6_tb2MLEazIORIzxI8RA',
 'updatedRange': "'工作表1'!A12:D21",
 'updatedRows': 10,
 'updatedColumns': 3,
 'updatedCells': 30}